In [3]:
%matplotlib widget

import argparse
import sys
import os

import matplotlib.pyplot as plt
import numpy as np

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import pybrams

In [6]:
def main(args: argparse.Namespace) -> int:
    
    pybrams.enable_cache()

    if args.clear_cache:

        pybrams.clear_cache()

    if not args.systems:

        args.systems = [system.system_code for system in pybrams.systems.all().values()]

    if args.command == "cams":

        cams = pybrams.optical.cams.CAMS(args)
        cams.load()

        for number in args.numbers:

            print(f"Trajectory {number} .. ")

            interval = cams.get_interval(number)

            files = pybrams.brams.files.get(interval, args.systems)

            for system_code, file_list in files.items():

                for file in file_list:

                    print(f"Processing {system_code} ..")

                    file.clean()

                    speed, equiv_range = cams.get_speed_range(number, file.location())

                    meteor = pybrams.event.meteor.Meteor()
                    i_meteor, q_meteor, times_meteor, sample_start, sample_end = meteor.extract_iq(interval.start, interval.end, file)
                    meteor_signal = file.signal.data[sample_start : sample_end + 1]

                    meteor_ampl = np.sqrt(i_meteor**2 + q_meteor**2)

                    plt.figure()
                    plt.plot(times_meteor, meteor_signal)
                    plt.xlabel("Time [s]")
                    plt.ylabel("Signal [ADU]")
                    plt.title(f"Meteor signal at {system_code}")
                    plt.grid()
                    plt.tight_layout()
                    plt.show()

                    plt.figure()
                    plt.plot(times_meteor, meteor_ampl)
                    plt.xlabel("Time [s]")
                    plt.ylabel("Amplitude [ADU]")
                    plt.title(f"Amplitude - Speed = {np.round(speed,2)} km/s - Equiv. range = {np.round(equiv_range,2)} km")
                    plt.grid()
                    plt.tight_layout()
                    plt.show()

                    index_t0, SNR, _ = pybrams.event.meteor.compute_t0_and_SNR(meteor_ampl, file.signal.samplerate, plot=True)


    if args.command == "brams":

        interval = pybrams.utils.interval.Interval.from_string(args.interval_str)

        files = pybrams.brams.files.get(interval, args.systems)

        for system_code, file_list in files.items():

                for file in file_list:

                    print(f"Processing {system_code} ..")

                    file.clean()

                    meteor = pybrams.event.meteor.Meteor()
                    i_meteor, q_meteor, times_meteor, sample_start, sample_end = meteor.extract_iq(interval.start, interval.end, file)
                    meteor_signal = file.signal.data[sample_start : sample_end + 1]

                    meteor_ampl = np.sqrt(i_meteor**2 + q_meteor**2)

                    plt.figure()
                    plt.plot(times_meteor, meteor_signal)
                    plt.xlabel("Time [s]")
                    plt.ylabel("Signal [ADU]")
                    plt.title(f"Meteor signal at {system_code}")
                    plt.grid()
                    plt.tight_layout()
                    plt.show()

                    plt.figure()
                    plt.plot(times_meteor, meteor_ampl)
                    plt.xlabel("Time [s]")
                    plt.ylabel("Amplitude [ADU]")
                    plt.title(f"Amplitude")
                    plt.grid()
                    plt.tight_layout()
                    plt.show()

                    index_t0, SNR, _ = pybrams.event.meteor.compute_t0_and_SNR(meteor_ampl, file.signal.samplerate, plot=True)                    
   
    return 0

In [ ]:
optical = False

class Args():
  
  clear_cache = False

  if optical:

    command = "cams"
    date = "2020-07-29"
    numbers = ['79'] 
    systems = ["BEOUDS_SYS001"]
  
  else:

    command = "brams"
    interval_str = "2020-07-29T23:13:59/2020-07-29T23:14:03"
    systems = ["BEOUDS_SYS001"]


args = Args()

main(args)